### Generación de archivos de medición 

In [16]:
from math import pi
import numpy as np
def format_with_fixed_digits(number,d):

    num = set_decimal_places(number,d)
    if d == 4:
        formatted_number = f'{num:+.4f}'
    elif d==3:
        formatted_number = f'{num:+.3f}'
    elif d==2:
        formatted_number = f'{num:+.2f}'
    elif d==1:
        formatted_number = f'{num:+.1f}'
    integer_part, decimal_part = formatted_number.split('.')
    integer_part = integer_part.zfill(2)
    # return f"{integer_part}.{decimal_part}"
    return f"{integer_part}.{decimal_part}E00"
def set_decimal_places(number, d):
    """
    Set the number of decimal places for a given number.

    :param number: The number to set the decimal places for.
    :param d: The desired number of decimal places.
    :return: The number with the specified number of decimal places.
    """
    if isinstance(number, (int, float)) and isinstance(d, int) and d >= 0:
        formatted_number = "{:.{precision}f}".format(number, precision=d)
        return float(formatted_number)
    else:
        raise ValueError("Invalid input. 'number' must be a number, and 'd' must be a non-negative integer.")

def count_digits(number):
    """
    Count the number of digits in a given number based on conditions.

    :param number: The number to count digits in.
    :return: The number of digits based on the specified conditions.
    """
    if isinstance(number, (int, float)):
        abs_number = abs(number)
        if abs_number >= 100:
            return 2
        elif abs_number >= 10:
            return 3
        elif abs_number >= 0:
            return 4
        else:
            return 0  # Handle cases where the number is 0 or less than 1
    else:
        raise ValueError("Invalid input. 'number' must be a number.")
    
    
    
# set_decimal_places(3.14156,5)
# format_with_fixed_digits(300.141516,2)

# rng = np.random.default_rng()
def generate_noise_values(A,n):
    fs = 20e3
    t = np.linspace(0, n/fs, n+2)
    # add noise to signal
    noise_power = A * 0.05
    i1_values =  rng.normal(scale=np.sqrt(noise_power), size=t.shape)
    return i1_values,t

import csv
import datetime
import numpy as np
from math import pi
rng = np.random.default_rng()


# Define the initial date and time
initial_datetime = datetime.datetime(2023, 2, 17, 13, 44, 4, 94 * 1000)  # Year, Month, Day, Hour, Minute, Second, Microsecond

# Define the CSV file path
# csv_file = "output.csv"
output_filename = "output.csv"
output_index = 0
total_rows_written = 0

def open_new_csv_file():
    global output_filename, output_index
    if output_index == 0:
        return open(output_filename, mode='w', newline='')
    else:
        # output_index += 1
        output_filename = f"output_{output_index:03d}.csv"
        return open(output_filename, mode='w', newline='')

# Function to generate sine wave values for U1 and I1
def generate_pure_sine_wave_values(A,f1,n):
    fs = 20e3
    t = np.linspace(0, n/fs, n+2)
    # Generate sine wave values for U1 and I1
    i1_values = A * np.sin(2*pi*(f1) * t)
    return [i1_values,t]
def generate_noise_values(A,n):
    fs = 20e3
    t = np.linspace(0, n/fs, n+2)
     # add noise to signal
    noise_power = A
    i1_values =  rng.normal(scale=np.sqrt(noise_power), size=t.shape)
    return i1_values,t

def generate_sine_wave_values(n):
    fs = 20e3
    # Define parameters for the sine waves
    amplitude_i1 = 3.5
    f1 = 50  # Adjust this as needed
    t = np.linspace(0, n/fs, n+2)
    
    # Generate sine wave values for U1 and I1
    i1_values =  0*np.sin(0*t)
    i1_values += amplitude_i1*generate_pure_sine_wave_values(1,f1,n)[0]
    i1_values += 0.1*amplitude_i1*generate_pure_sine_wave_values(1,66,n)[0]
    i1_values += 0.05*amplitude_i1*generate_pure_sine_wave_values(1,105,n)[0]
    i1_values += 0.01*amplitude_i1*generate_pure_sine_wave_values(1,141,n)[0]

    # add noise to signal
    # noise_power = amplitude_i1 * 0.000001
    noise_power = max(i1_values) * 0.001
    i1_values +=  rng.normal(loc=t[int(len(t)/2)],
    # i1_values +=  rng.normal(loc=0,
                             scale=np.sqrt(noise_power), size=t.shape)

    # i1_values +=  rng.normal(loc=t[int(len(t)/2)],
    #                          scale=np.sqrt(noise_power), size=t.shape)
    # ## Include freq components 
    # tmin=max(t)/2
    # tmax=0.75*max(t)
    # t_slice = np.where((t >= tmin) & (t <= tmax))

    # t_sliced = t[t_slice]   
    # # Add freq components after the half of the sample
    # # if i > num_rows/4:
    # i1_values_3 = 0.1*amplitude_i1 * np.sin(77*2*np.pi * t)
    # i1_values[t_slice]= i1_values[t_slice] +i1_values_3[t_slice]
    return 0, i1_values,t


# Write the sequence to a CSV file
# with open(csv_file, mode='w', newline='') as file:
with open_new_csv_file() as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["EventNo", "Date", "Time", "U1", "I1"])
    
    event_no = 1
    num_rows = int(5e6)
    _,i1_values,t = generate_sine_wave_values(num_rows)
    # i1_values = generate_pure_sine_wave_values(1,50,num_rows)

    # Define the number of digits according to max-min values
    m1 = max(i1_values)
    m2 = abs(min(i1_values))
    m3 = max([m1,m2])
    d = count_digits(m3)
    

    for i in range(num_rows):
        date = initial_datetime.strftime("%d-%b-%y")
        time = initial_datetime.strftime("%H:%M:%S.%f")[:-3]  # Format the timestamp
        # u1 = u1_values[i]
        u1 = i1_values[i]
        i1 = i1_values[i]

        # Format the U1 and I1 values with a fixed number of digits for the exponent
        # u1_formatted = format_with_fixed_exponent(u1, 0)
        # i1_formatted = format_with_fixed_exponent(i1, 0)
        u1_formatted = format_with_fixed_digits(u1,d)
        i1_formatted = format_with_fixed_digits(i1,d)
        

        # Write the row with "Date" and "Time" separated
        # writer.writerow([event_no, date, time, f"{u1:.2E}", f"{i1:.2E}"])
        # writer.writerow([event_no, date, time, f"{u1:+.3f}E+00", f"{i1:+.3f}E+00"])
        writer.writerow([event_no, date, time, u1_formatted, i1_formatted])

        
        # Increment the date every 20 events
        if (i + 1) % 20 == 0:
            initial_datetime += datetime.timedelta(milliseconds=1)
        # Increment the event number every 5600 steps
        if (i + 1) % 5600 == 0:
            event_no += 1
        # Check if it's time to create a new file
        total_rows_written += 1



        # if total_rows_written >= 1047201:
        if total_rows_written >= 1047200:
            output_index+=1
            file.close()
            file = open_new_csv_file()
            writer = csv.writer(file)
            writer.writerow(["EventNo", "Date", "Time", "U1", "I1"])
            total_rows_written = 0


# print(f"CSV file '{csv_file}' has been generated successfully.")
print(f"CSV files have been generated successfully.")



CSV files have been generated successfully.


In [11]:
_,a,_ = generate_sine_wave_values(num_rows)
k = int(1.5e3)
print(a[k])
print((format_with_fixed_digits(a[k],4)))

-3.726057904248455
-3.7261E00


### Prueba de diagnóstico

In [ ]:
%reset -f
import POST_proceso as pp
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal


# filepath=r"C:\GitHub\CLB_fft\Predictive_maintenance_module\output.csv"
filepath=r"C:\GitHub\CLB_fft\Predictive_maintenance_module\0.66_1323rpm_rodamiento_nuevo.csv"
# filepath=r"C:\ep_nvp\CLB_fft-main\NH-I\LCA\B3022002\B3022002.csv"

t,y = pp.ejecutar(filepath)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import detrend
from scipy.fftpack import fft 

import matplotlib.pyplot as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": "Helvetica",
# })

# Parameters
sampling_freq = int(20e3)  # Sampling frequency in Hz
duration = t[-1]  # Duration in seconds
composite_signal = np.array(y)
# Complex demodulation at 50Hz carrier frequency
carrier_freq = 50.0
demodulated_signal = composite_signal * np.exp(-2j * np.pi * carrier_freq * t)

# FFT of the original, demodulated, and high-pass filtered signals
fft_original = fft(composite_signal)
fft_demodulated =fft((demodulated_signal.real-np.mean(demodulated_signal.real)))

# Frequency axis for plotting
freq_axis = np.fft.fftfreq(len(t), 1.0 / sampling_freq)


In [ ]:
# plt.rcParams['figure.dpi'] = 150
# from IPython.display import set_matplotlib_formats
# matplotlib_inline.backend_inline.set_matplotlib_formats(  set_matplotlib_formats('svg'))
# Plot the instantaneous values
fig,(ax2)=plt.subplots(1, 1)
y1 =((fft_original))
y2 =((fft_demodulated))
k1 = 1/max(y1)
k2 = 1/max(y1)
N = len(t)

# yf1     =2.0/N * abs(y1[:N//2])
# yf2     =2.0/N * abs(y2[:N//2])


y1 = 10*np.log10(k1*y1)
y2 = 10*np.log10(k2*y2)
# Plot the FFTs
# fig,(ax1) = plt.subplots(2, 1)
ax2.plot(freq_axis, y1.real, label="FFT of Original Signal")
ax2.plot(freq_axis, y2.real, label="FFT of Demodulated Signal")
# ax2.plot(freq_axis, np.abs(fft_high_pass_filtered), label="FFT of High-Pass Filtered Signal")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
ax2.legend()



plt.tight_layout()
plt.ylim(-70,1)
plt.xlim(-1,200)
plt.grid(True,'both')
plt.show()


In [ ]:
from scipy.signal import find_peaks
y2=y2.real
y1=y1.real
noise_level=4*y1.mean()/y1.std()
ypk,prop = find_peaks(y1,height=noise_level,
                      prominence=10,
                      distance=50,
                      width=5)
# ypk,prop= find_peaks(y2,height=-40)
plt.plot(freq_axis,y1)
plt.plot(freq_axis[ypk], y1[ypk], "x")
# plt.xlim(49,51)
plt.plot(noise_level*np.ones(np.shape(y1)))
plt.xlim(0,150)

# plt.plot(np.zeros_like(y2), "--", color="gray")
plt.show()

In [ ]:
import FreqGen as fg

NonClassifiedFreq = list()
DiagnosticDict = dict()
KnownFreqDict = dict()

KnownFreqDict["GRID"] = fg.grid_frequencies(50,10)
KnownFreqDict["BRBAR"] = fg.broken_bar_frequencies(50,20,0.118+0.98*0.001)
# KnownFreqDict[] = 

for i in ypk:
    if freq_axis[i] >=0 :
        NonClassifiedFreq.append([freq_axis[i],abs(y1[i])])

DiagnosticDict['unclassified']=list()
for j in KnownFreqDict.keys():
    DiagnosticDict[j]=list()

tol = .1 #Hz
Nfreqs =len(NonClassifiedFreq)

for i in range(Nfreqs):
    fi =NonClassifiedFreq[i]
    fival = fi[0]

    for j in KnownFreqDict.keys():
        if np.any(np.abs(np.array(KnownFreqDict[j])-fival)<=tol):
            order = np.argmin(np.abs(np.array(KnownFreqDict[j])-fival))
            DiagnosticDict[j].append([fi,order])
        else:
            DiagnosticDict['unclassified'].append(fi)
print(DiagnosticDict['GRID'])
print(DiagnosticDict['BRBAR'])
# print(len(DiagnosticDict['unclassified'])/Nfreqs)
# print(GRID)
# print(DiagnosticDict['unclassified'])


In [ ]:
# %matplotlib notebook
plt.plot(freq_axis,y1)
plt.plot(freq_axis[ypk], y1[ypk], "x")
# plt.xlim(49,51)
plt.plot(noise_level*np.ones(np.shape(y1)))
plt.xlim(0,150)

# plt.plot(np.zeros_like(y2), "--", color="gray")

point = DiagnosticDict['BRBAR'][2][0]
point[-1] = -point[-1]
print(point)
ax = plt.gca()
radius = 15

ax.plot(point[0], point[1], 'o',
   # ms=radius * 2, mec='yellow', mfc='none', mew=2)
   ms=radius * 2, mec='red', mfc='none', mew=2)

ax.annotate('BRBAR', xy=point, xytext=(60, 60),
   textcoords='offset points',
   color='green', size='large',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.1,head_width=0.4,head_length=0.4',
      facecolor='b', shrinkB=radius * 1.2)
   )

plt.show()


In [ ]:
def generate_frequency_content(n, content_type):
    content = []
    content_formulas = {
        "grid_frequency": lambda i: i,
        "bearings_harmonics": lambda i: 2 * i,  # Example formula for bearings harmonics
        "eccentricity": lambda i: 3 * i,  # Example formula for eccentricity content
        # Add more content types and their formulas as needed
    }
    if content_type in content_formulas:
        formula = content_formulas[content_type]
        content = [formula(i) for i in range(1, n + 1)]
    else:
        print(f"Unsupported content type: {content_type}")
    return content

# Example usage:
n = 10
content_type = "grid_frequency"
result = generate_frequency_content(n, content_type)
print(result)
    

In [ ]:
def calculate_eccentricity_frequencies(fg, R, nd, s, p, nws):
    """
    fec: eccenticity frequency
    fg: electrical supply (grid) frequency
    R: the number of rotor bars
    s:  slip
    p: pole-pairs
    nd: ±1
    nws: 1, 3, 5, 7 
    """
    fecpp = fg * ((R + nd )*(1-s) / p + nws)
    fecpm = fg * ((R + nd )*(1-s) / p - nws)
    fecmp = fg * ((R - nd )*(1-s) / p + nws)
    fecmm = fg * ((R - nd )*(1-s) / p - nws)
    fc = R*fg
    return fecpp, fecpm , fc, fecmp, fecmm
def calculate_rotor_bar_frequencies(fg, s):
    frbp = fg * (1 + 2 * s)
    frbm = fg * (1 - 2 * s)
    return frbp, frbm
def calculate_bearing_frequencies(nb, frm):
    f0 = 0.4 * n * frm
    f1 = 0.6 * n * frm
    return f0, f1

# Example usage:
nb = 12  # Number of balls in the bearing (adjust as needed)
frm = 60  # Running speed frequency in Hz (typically grid frequency)

f0, f1 = calculate_bearing_frequencies(n, frm)
print(f"Bearing Frequency Component (f0): {f0} Hz")
print(f"Bearing Frequency Component (f1): {f1} Hz")

# Example usage:
fg = 50  # Grid frequency in Hz
s = 0.4  # Slip (adjust as needed)
k = 1  # 1,2,3,....
s = 0.025  # Slip
p = 1  # Number of poles
# Eccentricity vals
R = 48  # Eccentricity amplitude
n = 1  # +-1
nws = 3  # 1,3,5,7

frbp, frbm = calculate_rotor_bar_frequencies(fg, s)
print(f"Rotor Bar Frequency Component (frbp): {frbp} Hz")
print(f"Rotor Bar Frequency Component (frbm): {frbm} Hz")


# fecpp, fecpm, fc,fecmp,fecmm = calculate_eccentricity_frequencies(fg, R, n, s, p, nws)
# print(f"Eccentricity Frequency Component (fecpp): {fecpp} Hz")
# print(f"Eccentricity Frequency Component (fecpm): {fecpm} Hz")
# print(f"Eccentricity central Frequency Component (fc): {fc} Hz")

# print(f"Eccentricity Frequency Component (fecpp): {fecmm} Hz")
# print(f"Eccentricity Frequency Component (fecpm): {fecmp} Hz")

In [ ]:
import FreqGen as fg

GRID = fg.grid_frequencies(50,10)
BRBAR= fg.broken_bar_frequencies(50,10,1e-3)

print(np.any((np.array(GRID)-49.6)<=1))
np.argmin(abs(np.array(GRID)-169.6))



In [ ]:
import matplotlib
from matplotlib import rc
print(matplotlib.get_configdir())
sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist])

rc('text.latex', preamble=r'\usepackage{sfmath}')